# 0.InportLibraries

In [352]:
import numpy as np
import copy
from sklearn.neural_network import MLPRegressor
from numpy.random import *
from operator import itemgetter

# 1.InputFieldData

In [353]:
def sortPosInfo(labeled_poses,info_name):
    labeled_poses.sort(key=lambda x:x[1])
    sorted_poses = []
    for pos in labeled_poses:
        sorted_poses.append(pos[0])
    print("{0} = {1}".format(info_name,sorted_poses))
    return sorted_poses

In [354]:
def printFieldData(field):
    print("Field:\n----------")
    for row in field:
        print(row)
    print("----------")

In [355]:
def inputFieldData(text_path):
    f = open(text_path)
    line = f.readline()
    line_count = 1
    field_size = int(len(line)/2)
#     field = np.empty((0,field_size),int)
    field = []
    agent_poses = []
    goal_poses = []
    while line:
        last_idx = line.find("\n")
        field_row = []
        for i in range(0,last_idx,2):
            field_row.append(int(line[i]))
            if int(line[i]) % 2 == 0 and int(line[i]) != 0:
                agent_data = [[line_count-1,int(i/2)],int(line[i])]
                agent_poses.append(agent_data)
            if int(line[i]) % 2 == 1 and int(line[i]) != 1:
                goal_data = [[line_count-1,int(i/2)],int(line[i])]
                goal_poses.append(goal_data)
        field.append(field_row)
        line = f.readline()
        line_count += 1
    f.close()
    printFieldData(field)
    agent_poses = sortPosInfo(agent_poses,"agents_pos_list")
    goal_pos_list = sortPosInfo(goal_poses,"goal_pos_list")
    return field,agent_poses,goal_poses

In [356]:
init_field,agent_poses,goal_poses = inputFieldData("demoField.txt")
field_row = len(init_field[0])
field_col = len(init_field)

Field:
----------
[1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 3, 0, 0, 0, 7, 0, 4, 1]
[1, 0, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 5, 6, 0, 1, 0, 0, 2, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]
----------
agents_pos_list = [[4, 7], [1, 7], [4, 2]]
goal_pos_list = [[1, 1], [4, 1], [1, 5]]


# 2.DecideActions
### action={0:up, 1:right, 2:down, 3:left, 4:stay}

In [357]:
def moveToNextPos(pos,action):
    if action == 0:
        next_pos = [pos[0]-1,pos[1]]
    elif action == 1:
        next_pos = [pos[0],pos[1]+1]
    elif action == 2:
        next_pos = [pos[0]+1,pos[1]]
    elif action== 3:
        next_pos = [pos[0],pos[1]-1]
    else:
        next_pos = [pos[0],pos[1]]
    return next_pos

In [358]:
def buildNetwork(X_train,y,train):
    clf = MLPRegressor(solver="adam",random_state=1,max_iter=1000,hidden_layer_sizes=(20,50,10))
    print("X_train.shape = {0}, y_train.shape={1}".format(X_train.shape,y_train.shape))
    clf.fit(X_train,y_train)
    return clf

In [359]:
def checkNextField(action_data,poses,pred_actions,field,agent_num):
#     print("method_start")
    next_poses = [moveToNextPos(poses[i],pred_actions[i]) for i in range(len(poses))]
    next_pos = next_poses[agent_num]
    action_flag = True
    #行動先が行動可能かどうか判定
    if field[next_pos[0]][next_pos[1]] != 1: #行動先が壁でないとき
        for i in range(len(next_poses)):
            #エージェントの向きと真逆の方向へは移動できない
            prev_action = 4 # init
            if action_data:
                prev_action = action_data[-1][i]
            #次の目的地設定後1ステップ目は前回の移動方向を参照して真逆の方向への移動ができない制限を加える必要がある
#             elif all_best_action_data:
#                 prev_action = all_best_action_data[-1][-1][i]
            pred_action = pred_actions[i]
            if (prev_action == 0 and pred_action == 2) or (prev_action == 1 and pred_action == 3) or (prev_action == 2 and pred_action == 0) or (prev_action == 3 and pred_action == 1):
                action_flag = False
#                 print("a")
            #行動先に先約があるとき
            if agent_num == i: continue
            if next_pos == next_poses[i]:
                action_flag = False
#                 print("b")
            #他エージェントと交差するとき
            if next_pos == poses[i] and next_poses[i] == poses[agent_num]:
                action_flag = False
#                 print("c")
    else:
        action_flag = False
    return action_flag

In [360]:
def checkAchievingGoal(poses,goal_flags):
    for i in range(len(poses)):
        if poses[i] == goal_poses[i]:
            goal_flags[i] = True
    return goal_flags

In [390]:
def checkActions(action_data,pred_actions,field,poses,goal_flags):
    next_poses = []
    exit_flag = False
    goal_flags = checkAchievingGoal(poses,goal_flags)
    for i in range(len(pred_actions)):
        next_poses.append(moveToNextPos(poses[i],pred_actions[i]))
        if goal_flags[i]:
            pred_actions[i] = 5
    for i in range(len(next_poses)):
        if goal_flags[i]:
            continue
        #行動先が行動可能かどうか判定
        action_flag = checkNextField(action_data,poses,pred_actions,field,i)
        #行動不可能なとき，注目するエージェントの行動先を変更
        cnt = 0
        while action_flag == False:
            pred_actions[i] = int(pred_actions[i] + rand() *3 + 1) % 4
            action_flag = checkNextField(action_data,poses,pred_actions,field,i)
            cnt += 1
            if cnt > 100:
                exit_flag = True
                break
    return pred_actions,exit_flag

In [391]:
def predictActions(clf,field):
    X_test = np.asarray(field).reshape(1,field_row*field_col)
    actions = clf.predict(X_test)[0]
    for i in range(len(actions)):
        actions[i] = int(actions[i])
    if rand() < 0.2:
#         for i in range(len(actions)):
#             if rand() < 0.5:
#                 actions[i] = int(actions[i] + rand() * 3 + 1) % 4
        actions[i] = int(actions[i] + rand() * 3 + 1) % 4
    return actions

In [392]:
def updateField(poses,next_poses,field):
    for i in range(len(agent_poses)):
        pos = poses[i]
        next_pos = next_poses[i]
        #エージェントがいずれかの方向に進行する場合
        if pos != next_pos:
            #現在いる地点がいずれかのエージェントのゴール地点でないとき
            if field[pos[0]][pos[1]] % 2 == 0:
                field[pos[0]][pos[1]] = 0
            #現在いる地点がいずれかのエージェントのゴール地点のとき，現在地の値は変更しない
            
            #遷移先がいずれかのゴール地点でないとき
            if field[next_pos[0]][next_pos[1]] % 2 == 0:
                field[next_pos[0]][next_pos[1]] = i * 2 + 2
            #遷移先がいずれかのゴール地点のとき遷移先の値は変更しない

In [393]:
# FIND_ROOT_MAX_LOOP = 30
# train_data_cnt = 0
# def createTrainData(init_field,init_poses,clf,path_count,best_field_data,best_action_data,last_poses):
#     global train_data_cnt
#     field_data = [copy.deepcopy(init_field)]
#     field = copy.deepcopy(init_field)
#     poses = init_poses
#     action_data = []
#     goal_flags = [False for i in range(len(poses))]
#     create_flag = False
#     exit_flag = False
#     for i in range(FIND_ROOT_MAX_LOOP):
# #         print("agent_poses = {0}".format(poses))
#         if train_data_cnt > 3:
#             pred_actions = predictActions(clf,field)
#         else:
#             pred_actions = [int(rand()*4) for i in range(len(agent_poses))]
#         #予測したアクションに対してチェックを行い，壁や他エージェントにぶつかる挙動を変更する
#         actions,exit_flag = checkActions(action_data,pred_actions,field,poses,goal_flags)
#         if exit_flag:
#             break
#         #タイムステップ毎のアクションを保持するaction_dataの末尾に次の行動を追加
#         action_data.append(actions)
#         #actionsを元に次のfieldへと遷移させる
#         next_poses = []
#         for j in range(len(agent_poses)):
#             next_poses.append(moveToNextPos(poses[j],actions[j]))
#         updateField(poses,next_poses,field)
#         field_data.append(copy.deepcopy(field))
#         poses = next_poses
# #         printFieldData(field)
#         if goal_flags[0] and goal_flags[1] and goal_flags[2]:
#             action_data.append([4,4,4])
#             np_field_data = np.asarray(field_data)
#             np_action_data = np.asarray(action_data)
#             if path_count > np_field_data.shape[0] - 5:
#                 path_count = np_field_data.shape[0]
#                 clf = buildNetwork(np_field_data.reshape(np_field_data.shape[0],np_field_data.shape[1]*np_field_data.shape[2]),np_action_data)
#                 train_data_cnt += 1
#                 best_field_data = copy.deepcopy(field_data)
#                 best_action_data = copy.deepcopy(action_data)
#                 last_poses = copy.deepcopy(poses)
#                 create_flag = True
#             break
#     return last_poses,clf,path_count,best_field_data,best_action_data,create_flag

In [396]:
FIND_ROOT_MAX_LOOP = 100
train_data_cnt = 0
def createTrainData(init_field,init_poses,clf,path_count,best_field_data,best_action_data,last_poses):
    global train_data_cnt
    field_data = [copy.deepcopy(init_field)]
    field = copy.deepcopy(init_field)
    poses = init_poses
    action_data = []
    goal_flags = [False for i in range(len(poses))]
    create_flag = False
    exit_flag = False
    first_goal_flag = False
    dummy_first_goal_idx = -1
    dummy_first_goal_poses = []
    for i in range(FIND_ROOT_MAX_LOOP):
#         if clf != None:
        if train_data_cnt >6:
            pred_actions = predictActions(clf,field)
        else:
            pred_actions = [int(rand()*4) for i in range(len(agents_pos_list))]
        #予測したアクションに対してチェックを行い，壁や他エージェントにぶつかる挙動を変更する
        actions,exit_flag = checkActions(action_data,pred_actions,field,poses,goal_flags)
        if (goal_flags[0] or goal_flags[1] or goal_flags[2]) and first_goal_flag == False:
            first_goal_flag = True
            dummy_first_goal_idx = len(action_data)
            dummy_first_goal_poses = copy.deepcopy(poses)
        if exit_flag:
            break
        #タイムステップ毎のアクションを保持するaction_dataの末尾に次の行動を追加
        action_data.append(actions)
        #actionsを元に次のfieldへと遷移させる
        next_poses = []
        for j in range(len(agents_pos_list)):
            next_poses.append(moveToNextPos(poses[j],actions[j]))
        updateField(poses,next_poses,field)
        field_data.append(copy.deepcopy(field))
        poses = next_poses
        if goal_flags[0] and goal_flags[1] and goal_flags[2]:
            action_data.append([4,4,4])
            np_field_data = np.asarray(field_data)
            np_action_data = np.asarray(action_data)
            if path_count > np_field_data.shape[0] - 5:
                path_count = np_field_data.shape[0]
                clf = buildNetwork(np_field_data.reshape(np_field_data.shape[0],np_field_data.shape[1]*np_field_data.shape[2]),np_action_data)
#                 printFieldData(field_data,action_data)
                train_data_cnt += 1
                best_field_data = copy.deepcopy(field_data)
                print("dummy:{0}".format(dummy_first_goal_idx))
                global first_goal_field_idx
                global first_goal_field_poses
                first_goal_field_idx = dummy_first_goal_idx
                first_goal_field_poses = copy.deepcopy(dummy_first_goal_poses)
                best_action_data = copy.deepcopy(action_data)
                last_poses = copy.deepcopy(poses)
                create_flag = True
            break
#     for i in range(30):
#         print(field_data[i])
#         print("action = {0} \n".format(action_data[i]))
#     print("--------------------------------------------------")
    return  last_poses,clf,path_count,best_field_data,best_action_data,create_flag

In [397]:
def runCreatingPath(agent_poses,init_field):
    clf = None
    path_count = 1e+7
    best_field_data = []
    best_action_data = []
    last_poses = []
    create_flag = False
    cnt = 0
    while train_data_cnt < 10:
        init_poses = agent_poses
        last_poses,clf,path_count,best_field_data,best_action_data,create_flag = createTrainData(init_field,init_poses,clf,path_count,best_field_data,best_action_data,last_poses)
    return best_field_data,best_action_data,last_poses